# Wildfire Hazard: Data Retrieval/Prep 1 - Area Boundary, Imagery & DEM

Program: wildfire_prep.ipynb  
Programmer: Brian Gauthier  
Purpose: This notebook retrieves and prepares data for wildfire hazard analysis  
Date: May 2, 2025

### Import Python Modules

In [1]:
import arcpy
import ee
import geemap
import json
import os
import requests
import tqdm
import zipfile

### Authenticate and Initialize Google Earth Engine and Geemap

In [2]:
#ee.Authenticate()
ee.Initialize(project='your-earthengine-project-name')
geemap.ee_initialize()

### Set Paths

In [3]:
# Get project directory
aprx = arcpy.mp.ArcGISProject("CURRENT")
gis_dir = os.path.dirname(aprx.filePath)
project_dir = os.path.dirname(gis_dir)

# Data Directory structure
raw_dir = os.path.join(project_dir, "data", "raw")
extract_dir = os.path.join(project_dir, "data", "extracted")
proc_dir = os.path.join(project_dir, "data", "processed")

# Create folders if they don't exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(extract_dir, exist_ok=True)
os.makedirs(proc_dir, exist_ok=True)

### Setup Map

In [4]:
map = geemap.Map()
map.setCenter(-63.106018, 44.871443, 7)

### Load Administrative Boundaries Dataset

In [5]:
dataset = ee.FeatureCollection("FAO/GAUL/2015/level2")

### Investigate the Dataset

In [ ]:
# Get the first feature in the dataset
first_feature = dataset.first()

# Get the properties (field names)
field_names = first_feature.propertyNames().getInfo()
print(field_names)

In [ ]:
# Get a list of unique Province level values (ADM1 in this case)
adm1_names = dataset.aggregate_array('ADM1_NAME').getInfo()
print(set(adm1_names))  # Convert to set to remove duplicates

In [ ]:
# Get a list of unique County level values (ADM2_NAME in this case)
adm2_names = dataset.aggregate_array('ADM2_NAME').getInfo()
print(set(adm2_names))  # Convert to set to remove duplicates

### Filter for Area of Inteterest (HRM)

In [6]:
# Filter for Nova Scotia
ns = dataset.filter(ee.Filter.eq('ADM1_NAME', 'Nova Scotia / Nouvelle-Écosse'))

print(ns.size().getInfo())

18


In [7]:
# Filter for Halifax, Nova Scotia
hali = ns.filter(
    ee.Filter.eq('ADM2_NAME', 'Halifax')
)

# Check the size of the filtered collection
print(hali.size().getInfo())

1


In [12]:
# Add layer to map
#map.addLayer(hali.geometry(), {'color': 'red'}, "HRM Boundary")
# Assume hali is an ee.Feature or ee.FeatureCollection
# If it's a FeatureCollection:
styled = hali.style(**{
    'color': 'black',        # outline color
    'fillColor': '00000000'  # fully transparent fill using 8-digit hex (last two are alpha)
})

map.addLayer(styled, {}, "HRM Boundary")


### Remove non-contiguous entities (Sable Island)

In [ ]:
# Define Sable Island's approximate boundary as a buffer around its coordinates
sable_island = ee.Geometry.Point([-59.9061, 43.9336]).buffer(100000)  # 100km buffer

# Get the geometry of Halifax
hali_geom = hali.geometry()

# Subtract Sable Island from Halifax's geometry
hali_fixed_geom = hali_geom.difference(sable_island)

# Convert back to a FeatureCollection
hali_fixed = ee.FeatureCollection(ee.Feature(hali_fixed_geom))

In [ ]:
# Add to the map
map.addLayer(hali_fixed, {'color': 'blue'}, "HRM (Without Sable Island)")

### Split the HRM into 4 equal sections based on longitude

In [ ]:
# Get the bounds of the hali_fixed area
hali_bounds = hali_fixed.bounds()

# Print the bounds (coordinates of the bounding box)
bounds = hali_bounds.getInfo()
print("Bounds of hali_fixed:", bounds)

# Coordinates of the bounding box
coordinates = bounds['coordinates'][0]

# Extract all the longitudes and latitudes
longitudes = [coord[0] for coord in coordinates]
latitudes = [coord[1] for coord in coordinates]

# Calculate the minimum and maximum longitudes and latitudes
min_long = min(longitudes)
max_long = max(longitudes)
min_lat = min(latitudes)
max_lat = max(latitudes)

# Calculate the step size for splitting the longitude range into four equal sections
step = (max_long - min_long) / 4

# Define the three split longitudes (boundaries) to create four equal sections
split_longitude_1 = min_long + step
split_longitude_2 = min_long + 2 * step
split_longitude_3 = min_long + 3 * step

# Print the split longitudes
print("Split Longitudes:", split_longitude_1, split_longitude_2, split_longitude_3)

# Define the four new polygons by cutting the original polygon at the three split longitudes
# West region (1st section)
west_region = ee.Geometry.Polygon([
    [
        [min_long, min_lat],                # Southwest corner
        [split_longitude_1, min_lat],       # Southeast corner
        [split_longitude_1, max_lat],       # Northeast corner
        [min_long, max_lat],                # Northwest corner
        [min_long, min_lat]                 # Closing back to Southwest corner
    ]
])

# Centre-West region (2nd section)
centre_west_region = ee.Geometry.Polygon([
    [
        [split_longitude_1, min_lat],       # Southwest corner
        [split_longitude_2, min_lat],       # Southeast corner
        [split_longitude_2, max_lat],       # Northeast corner
        [split_longitude_1, max_lat],       # Northwest corner
        [split_longitude_1, min_lat]        # Closing back to Southwest corner
    ]
])

# Centre-East region (3rd section)
centre_east_region = ee.Geometry.Polygon([
    [
        [split_longitude_2, min_lat],       # Southwest corner
        [split_longitude_3, min_lat],       # Southeast corner
        [split_longitude_3, max_lat],       # Northeast corner
        [split_longitude_2, max_lat],       # Northwest corner
        [split_longitude_2, min_lat]        # Closing back to Southwest corner
    ]
])

# East region (4th section)
east_region = ee.Geometry.Polygon([
    [
        [split_longitude_3, min_lat],       # Southwest corner
        [max_long, min_lat],                # Southeast corner
        [max_long, max_lat],                # Northeast corner
        [split_longitude_3, max_lat],       # Northwest corner
        [split_longitude_3, min_lat]        # Closing back to Southwest corner
    ]
])

In [ ]:
# Add the four regions to the map with new names
map.addLayer(west_region, {'color': 'blue'}, 'West Region')
map.addLayer(centre_west_region, {'color': 'red'}, 'Centre-West Region')
map.addLayer(centre_east_region, {'color': 'green'}, 'Centre-East Region')
map.addLayer(east_region, {'color': 'yellow'}, 'East Region')

In [ ]:
# Clip the regions by intersecting them with the HRM boundary
west_clip = west_region.intersection(hali_fixed)
centre_west_clip = centre_west_region.intersection(hali_fixed)
centre_east_clip = centre_east_region.intersection(hali_fixed)
east_clip = east_region.intersection(hali_fixed)

In [ ]:
# Add the clipped regions to the map
map.addLayer(west_clip, {'color': 'blue'}, 'West Clipped')
map.addLayer(centre_west_clip, {'color': 'red'}, 'Centre-West Clipped')
map.addLayer(centre_east_clip, {'color': 'green'}, 'Centre-East Clipped')
map.addLayer(east_clip, {'color': 'yellow'}, 'East Clipped')

### Pull, Filter, Clip & Export Sentinel-2 Satellite Imagery

In [ ]:
# load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(centre_west_clip) \
    .filterDate('2024-06-01', '2024-10-31')

# Load cloud probability dataset
clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(centre_west_clip) \
    .filterDate('2024-06-01', '2024-10-31')

# Function to mask clouds based on cloud probability (< 30%)
def mask_clouds(image):
    cloud_prob = clouds.filter(ee.Filter.equals('system:index', image.get('system:index'))).first().select('probability')
    mask = cloud_prob.lt(30)  # Keep pixels with less than 30% cloud probability
    return image.updateMask(mask)

# Apply the loud mask to the collection
s2_clean = s2.map(mask_clouds)

# Reduce collection to a single image
s2_filtered = s2_clean.median()

# Clip the image to centre_west_clip
s2_clipped = s2_filtered.clip(centre_west_clip)

# Visualization parameters for RGB (Red, Green, Blue)
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']  # Red, Green, Blue bands
}

In [ ]:
# Add the image to the map
map.addLayer(s2_clipped, vis_params, 'CW Sentinel-2 Image (Clipped)')

In [ ]:
# Export the clipped image to Google Drive using this geometry
task = ee.batch.Export.image.toDrive(
    image=s2_clipped,
    description='Sentinel_2_Clipped',
    folder='EarthEngineExports',  # Folder in Google Drive
    fileNamePrefix='sentinel_2_clipped',
    region=centre_west_clip,
    scale=10,
    fileFormat='GeoTIFF'
)

# Start the export task
task.start()
print(task.status())

### Mosaic the exported sentinel-2 imagery (tif) files back together

In [ ]:
# Define input and output paths
s2_dir = r"D:\Dropbox\COGS\Capstone\data\processed\EE_Exports"
s2_mosaic_dir = os.path.dirname(s2_dir)
s2_mosaic_fn = "sentinel_2_mosaic.tif"
s2_mosaic_path = os.path.join(s2_mosaic_dir, s2_mosaic_fn)

# Collect all .tif files in the input folder
tif_files = [
    os.path.join(s2_dir, f)
    for f in os.listdir(s2_dir)
    if f.endswith(".tif")
]

# Iterate through TIF files and check the number of bands
for tif in tif_files:
    raster = arcpy.Raster(tif)
    print(f"{tif} has {raster.bandCount} bands")

# Mosaic to new raster
arcpy.management.MosaicToNewRaster(
    input_rasters=tif_files,
    output_location=s2_mosaic_dir,
    raster_dataset_name_with_extension=s2_mosaic_fn,
    coordinate_system_for_the_raster=arcpy.SpatialReference(4326),
    pixel_type="32_BIT_FLOAT",
    number_of_bands=16,
    mosaic_method="FIRST",
    mosaic_colormap_mode="MATCH"
)

print(f"Mosaic created at: {s2_mosaic_path}")

### Retrieve LiDAR DEM

In [ ]:
# Define the URL for DEM and file name/path for downloading
dem_url = "https://www.halifax.ca/opendata/files/HRM_LiDAR_DEM/HRM_LiDAR_DEM_2018_2m_wgs84.zip"
dem_zip_path = os.path.join(raw_dir, os.path.basename(dem_url))

In [ ]:
# Make request with streaming
response = requests.get(dem_url, stream=True)
total_size = int(response.headers.get('content-length', 0))

# Download with progress bar
if response.status_code == 200:
    with open(dem_zip_path, 'wb') as f, tqdm.tqdm(
        desc="Downloading",
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
                bar.update(len(chunk))
    print("Download complete.")
else:
    print("Download failed with status code:", response.status_code)

In [ ]:
# Extract the ZIP file
try:
    with zipfile.ZipFile(dem_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extraction complete. Files extracted to {extract_dir}")
except zipfile.BadZipFile:
    print("Error: The file is not a valid zip file.")
except Exception as e:
    print(f"An error occurred during extraction: {str(e)}")

### Export EE Geometry to Shapefile

In [ ]:
# Define Earth Engine FeatureCollection & shapefile path
centre_west_fc = ee.FeatureCollection(ee.Feature(centre_west_clip))
centre_west_shp_path = os.path.join(proc_dir, "centre_west.shp")

In [ ]:
# Export the FeatureCollection to a shapefile (geemap is key here to keep things local)
geemap.ee_to_shp(centre_west_fc, centre_west_shp_path)

# Confirm shapefile creation
print(f"Shapefile created at: {centre_west_shp_path}")

## Transform & Project Layers to NAD83 CSRS UTM Zone 20N, Clip to Study Area

In [ ]:
# Set ArcPy Workspace & overwrite options
arcpy.env.workspace = proc_dir
arcpy.env.overwriteOutput = True

In [ ]:
# Set input/output paths
centre_west_shp_utm_path = os.path.join(proc_dir, "centre_west_clip_utm.shp")
dem_path = os.path.join(extract_dir, "HRM_LiDAR_DEM_2018_2m_wgs84.tif")
dem_utm_path = os.path.join(proc_dir, "dem_centre_west_utm.tif")
dem_utm_clipped_path = os.path.join(proc_dir, "dem_centre_west_utm_clipped.tif")
s2_mosaic_utm_path = os.path.join(proc_dir, "sentinel_2_mosaic_utm.tif")
s2_mosaic_utm_clipped_path = os.path.join(proc_dir, "sentinel_2_mosaic_utm_clipped.tif")

### Centre West Shapefile

In [ ]:
# Check centre west shapefile crs before projecting
desc_centre_west_shp = arcpy.Describe(centre_west_shp_path)
print(desc_centre_west_shp.spatialReference.name)

In [ ]:
# Determine available geodetic transformations from WGS 1984 to NAD83 (CSRS)
# EPSG:4326 = WGS 1984
# EPSG:4617 = NAD83(CSRS)

arcpy.ListTransformations(
    arcpy.SpatialReference(4326),
    arcpy.SpatialReference(4617)
)


In [ ]:
# Transform and project centre_west_shapefile (HRM Boundary Subset) to NAD83 CSRS UTM Zone 20N
arcpy.management.Project(
    in_dataset=centre_west_shp_path,
    out_dataset=centre_west_shp_utm_path,
    out_coor_system=arcpy.SpatialReference(2961),
    transform_method="WGS_1984_(ITRF00)_To_NAD_1983 + NAD_1983_To_NAD_1983_CSRS_4"
)


### DEM

In [ ]:
# Check DEM crs before projecting
desc_dem = arcpy.Describe(dem_path)
print(desc_dem.spatialReference.name)

In [ ]:
# Project DEM to NAD83 CSRS UTM Zone 20N
arcpy.management.ProjectRaster(
    in_raster=dem_path,
    out_raster=dem_utm_path,
    out_coor_system=arcpy.SpatialReference(2961),
    resampling_type="BILINEAR",
    cell_size="2",
)

In [ ]:
# Clip reprojected DEM using the reprojected shapefile
arcpy.management.Clip(
    in_raster=dem_utm_path,
    out_raster=dem_utm_clipped_path,
    in_template_dataset=centre_west_shp_utm_path,
    clipping_geometry="ClippingGeometry",
    maintain_clipping_extent="MAINTAIN_EXTENT"
)

### Sentinel-2 Mosaic

In [ ]:
# Check mosaic crs before projecting
desc_mosaic = arcpy.Describe(s2_mosaic_path)
print(desc_mosaic.spatialReference.name)

In [ ]:
# Define the projection to WGS 1984 (EPSG:4326)
arcpy.management.DefineProjection(s2_mosaic_path, arcpy.SpatialReference(4326))

In [ ]:
# Reproject the raster to NAD83 CSRS UTM Zone 20N (EPSG:2961)
arcpy.management.ProjectRaster(
    in_raster=s2_mosaic_path,
    out_raster=s2_mosaic_utm_path,
    out_coor_system=arcpy.SpatialReference(2961), 
    resampling_type="BILINEAR", 
)

In [ ]:
# Clip sentinel-2 mosaic using the reprojected shapefile
arcpy.management.Clip(
    in_raster=s2_mosaic_utm_path,
    out_raster=s2_mosaic_utm_clipped_path,
    in_template_dataset=centre_west_shp_utm_path,
    clipping_geometry="ClippingGeometry",
    maintain_clipping_extent="MAINTAIN_EXTENT"
)